In [159]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import subprocess

%matplotlib inline

In [150]:
medication = pd.read_csv('output/medications/data.csv')
medication['CONDITION'] = medication.variable.apply(lambda x: re.sub(r"(\d)", "", x))
medication = medication[['CODI_ATC', 'NOMBRE', 'CONDITION']]
medication['id'] = medication.CODI_ATC.str.extract('(...)')

In [151]:
atc = pd.read_csv('data/ATC.csv')
all = pd.merge(medication, atc, on=['id'])
all = all[all.CONDITION != 'ENFERMEDADES_E']
all = all[all.CONDITION != 'ANTICONCEPTIVO']
all['COUNT'] = 1

In [155]:
gr = all.groupby(['CODI_ATC', 'NOMBRE', 'CONDITION', 'id', 'text', 'parent'])
ag = gr.agg('count').reset_index()
ag = ag.sort_values(['CONDITION', 'COUNT'], ascending=True)
tb = ag[['CODI_ATC', 'NOMBRE', 'CONDITION', 'text', 'COUNT']]
tb.columns = ['ATC_CODE', 'DRUG NAME', 'REPORTED CONDITION', 'CONDITION DRUG TREATS', 'COUNTS']

In [158]:
tb.to_csv('../output/conditions.csv', index=False)

In [161]:
template = r'''\documentclass{{standalone}}
\usepackage{{booktabs}}
\begin{{document}}
{}
\end{{document}}
'''

filename = 'figures/out.tex'
pdffile = 'figures/out.pdf'
outname = 'figures/out.png'

with open(filename, 'wb') as f:
    f.write(bytes(template.format(df.to_latex(index=False)),'UTF-8'))

subprocess.call(['pdflatex', filename])
subprocess.call(['convert', '-density', '300', pdffile, '-quality', '90', outname])

1